# How to work with raw csv EEG record in MNE

How to raw EEG record in csv format in MNE. Add channel names, sampling frequency and montage. Then save for EEGLAB

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib notebook

# In case you prefer svg plotting
# %matplotlib inline
# %config InlineBackend.figure_format = 'svg'

import mne

In [22]:
mne.__version__

'1.3.1'

In [2]:
from mne.channels.layout import _find_topomap_coords

Given csv with EEG record. Rows are channels. Columns are data samples. Channel names are in separate file

In [3]:
raw_nums = pd.read_csv('matrix_eeg_stack.csv', header=None, sep='\s')

/tmp/ipykernel_7316/2007279127.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  raw_nums = pd.read_csv('matrix_eeg_stack.csv', header=None, sep='\s')


In [4]:
raw_nums.shape

(31, 423150)

In [5]:
raw_nums.head(1)

,0,1,2,3,4,5,6,7,8,9,...,423140,423141,423142,423143,423144,423145,423146,423147,423148,423149
0,-0.000023,-0.000022,-0.000021,-0.000021,-0.000022,-0.000023,-0.000025,-0.000028,-0.00003,-0.000033,...,-0.000044,-0.000045,-0.000044,-0.000043,-0.000042,-0.00004,-0.000038,-0.000036,-0.000035,-0.000035


We know actual channel names

In [6]:
channels_to_use = [
    'Fp1',
    'Fpz',
    'Fp2',
    'F3',
    'Fz',
    'F4',
    'F7',
    'F8',
    'FC3',
    'FCz',
    'FC4',
    'FT7',
    'FT8',
    'C3',
    'Cz',
    'C4',
    'CP3',
    'CPz',
    'CP4',
    'P3',
    'Pz',
    'P4',
    'TP7',
    'TP8',
    'T3',
    'T4',
    'T5',
    'T6',
    'O1',
    'Oz',
    'O2'
]

## Save Montage for EEGLAB

In [7]:
ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')
info = mne.io.meas_info.create_info(channels_to_use, sfreq=500, ch_types="eeg")

In [8]:
raw = mne.io.RawArray(raw_nums.values, info)

Creating RawArray with float64 data, n_channels=31, n_times=423150
    Range : 0 ... 423149 =      0.000 ...   846.298 secs
Ready.


In [9]:
raw.to_data_frame().shape

(423150, 32)

In [10]:
ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')
raw.set_montage(ten_twenty_montage);

In [14]:
def save_montage_xyz(montage, path):
    rows = []
    for ch, coords in montage.get_positions()['ch_pos'].items():
        rows.append([ch, coords[0], coords[1], coords[2]])
#     haven't figured out how to use it in EEGLAB to scale topomaps
#     for ch in ['lpa', 'rpa', 'nasion']:
#         coords = montage.get_positions()[ch]
#         rows.append([ch, coords[0], coords[1], coords[2]])
    res = ''
    for row in rows:
        res += f'{row[0]} {row[1]:.4f} {row[2]:.4f} {row[3]:.4f}\n'
    with open(path, 'w') as f:
        f.write(res)

In [15]:
montage = raw.info.get_montage()

In [16]:
save_montage_xyz(montage, 'test_eeg.locs')

## Save EDF

Forked from

https://gist.github.com/skjerns/bc660ef59dca0dbd53f00ed38c42f6be

In [17]:
import pyedflib # pip install pyedflib
from pyedflib import highlevel # new high-level interface
from pyedflib import FILETYPE_BDF, FILETYPE_BDFPLUS, FILETYPE_EDF, FILETYPE_EDFPLUS
from datetime import datetime, timezone, timedelta
import mne
import os

In [18]:
from datetime import timezone

In [19]:
def _stamp_to_dt(utc_stamp):
    """Convert timestamp to datetime object in Windows-friendly way."""
    if 'datetime' in str(type(utc_stamp)): return utc_stamp
    # The min on windows is 86400
    stamp = [int(s) for s in utc_stamp]
    if len(stamp) == 1:  # In case there is no microseconds information
        stamp.append(0)
    return (datetime.fromtimestamp(0, tz=timezone.utc) +
            timedelta(0, stamp[0], stamp[1]))  # day, sec, μs


def write_mne_edf(mne_raw, fname, picks=None, tmin=0, tmax=None, 
                  overwrite=False):
    """
    Saves the raw content of an MNE.io.Raw and its subclasses to
    a file using the EDF+/BDF filetype
    pyEDFlib is used to save the raw contents of the RawArray to disk
    Parameters
    ----------
    mne_raw : mne.io.Raw
        An object with super class mne.io.Raw that contains the data
        to save
    fname : string
        File name of the new dataset. This has to be a new filename
        unless data have been preloaded. Filenames should end with .edf
    picks : array-like of int | None
        Indices of channels to include. If None all channels are kept.
    tmin : float | None
        Time in seconds of first sample to save. If None first sample
        is used.
    tmax : float | None
        Time in seconds of last sample to save. If None last sample
        is used.
    overwrite : bool
        If True, the destination file (if it exists) will be overwritten.
        If False (default), an error will be raised if the file exists.
    """
    if not issubclass(type(mne_raw), mne.io.BaseRaw):
        raise TypeError('Must be mne.io.Raw type')
    if not overwrite and os.path.exists(fname):
        raise OSError('File already exists. No overwrite.')
        
    # static settings
    has_annotations = True if len(mne_raw.annotations)>0 else False
    if os.path.splitext(fname)[-1] == '.edf':
        file_type = FILETYPE_EDFPLUS if has_annotations else FILETYPE_EDF
        dmin, dmax = -32768, 32767 
    else:
        file_type = FILETYPE_BDFPLUS if has_annotations else FILETYPE_BDF
        dmin, dmax = -8388608, 8388607
    
    print('saving to {}, filetype {}'.format(fname, file_type))
    sfreq = mne_raw.info['sfreq']
    date = _stamp_to_dt(mne_raw.info['meas_date'])
    
    if tmin:
        date += timedelta(seconds=tmin)
    # no conversion necessary, as pyedflib can handle datetime.
    #date = date.strftime('%d %b %Y %H:%M:%S')
    first_sample = int(sfreq*tmin)
    last_sample  = int(sfreq*tmax) if tmax is not None else None

    
    # convert data
    channels = mne_raw.get_data(picks, 
                                start = first_sample,
                                stop  = last_sample)
    
    # convert to microvolts to scale up precision
    channels *= 1e6

    # set conversion parameters
    n_channels = len(channels)
    
    # create channel from this   
    try:
        f = pyedflib.EdfWriter(fname,
                               n_channels=n_channels, 
                               file_type=file_type)
        
        channel_info = []
        
        ch_idx = range(n_channels) if picks is None else picks
        keys = list(mne_raw._orig_units.keys())
        for i in ch_idx:
            try:
                ch_dict = {'label': mne_raw.ch_names[i], 
                           'dimension': mne_raw._orig_units[keys[i]], 
                           'sample_rate': mne_raw._raw_extras[0]['n_samps'][i], 
                           'physical_min': mne_raw._raw_extras[0]['physical_min'][i], 
                           'physical_max': mne_raw._raw_extras[0]['physical_max'][i], 
                           'digital_min':  mne_raw._raw_extras[0]['digital_min'][i], 
                           'digital_max':  mne_raw._raw_extras[0]['digital_max'][i], 
                           'transducer': '', 
                           'prefilter': ''}
            except:
                ch_dict = {'label': mne_raw.ch_names[i], 
                           'dimension': 'uV', 
                           'sample_rate': sfreq, 
                           'physical_min': channels.min(), 
                           'physical_max': channels.max(), 
                           'digital_min':  dmin, 
                           'digital_max':  dmax, 
                           'transducer': '', 
                           'prefilter': ''}
        
            channel_info.append(ch_dict)
        # f.setPatientCode(mne_raw._raw_extras[0]['subject_info'].get('id', '0'))
        f.setPatientCode('')
        # f.setPatientName(mne_raw._raw_extras[0]['subject_info'].get('name', 'noname'))
        f.setPatientName('')
        f.setTechnician('mne-gist-save-edf-skjerns')
        f.setSignalHeaders(channel_info)
        f.setStartdatetime(date)
        f.writeSamples(channels)
        for annotation in mne_raw.annotations:
            onset = annotation['onset']
            duration = annotation['duration']
            description = annotation['description']
            f.writeAnnotation(onset, duration, description)
        
    except Exception as e:
        raise e
    finally:
        f.close()    
    return True

In [20]:
raw.set_meas_date(datetime.utcnow().replace(tzinfo=timezone.utc));

In [ ]:
write_mne_edf(raw, 'test_eeg.edf', overwrite=True)

## How to read in EEGLAB

```matlab
EEG = pop_biosig('test_eeg.edf')
EEG.chanlocs = readlocs('test_eeg.locs', 'filetype', 'custom', 'format', {'labels', 'X', 'Y', 'Z'})
```